In [69]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
from sklearn import cross_validation
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn import metrics, preprocessing
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [70]:
data = pd.read_csv('train.csv')

In [71]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [72]:
##drop passengerID and Name as these will not be much help in classification
data.drop(data.columns[[0, 3]], axis=1, inplace=True)

In [73]:
##Dummy code gender as 1 for females and 0 for males
data['Sex'] = data['Sex'].map({'female': 1, 'male': 0})


In [74]:
##fill all columns that have NaN or no values with the mean for that column
data = data.fillna(data.mean())

In [75]:
## for the string features, fill empty cells with 0 as their is no mean to be computed yet to allow for easier encoding 
data = data.fillna(0)

In [76]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,0,22,1,0,A/5 21171,7.2500,0,S
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,0,S
3,1,1,1,35,1,0,113803,53.1000,C123,S
4,0,3,0,35,0,0,373450,8.0500,0,S


In [77]:
## one hot encode all of the categorical values Ticket, Cabin, Embarked
df_dummy = pd.get_dummies(data, prefix=['Ticket', 'Cabin', 'Embarked'])

In [111]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,7.9250,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [78]:
##merge the one hot encoded columns with the other data frame
data = pd.merge(data, df_dummy)

In [110]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,7.9250,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [79]:
## drop the non-one-hot encoded values 
data.drop(data.columns[[ 6, 8, 9]], axis=1, inplace=True)

In [80]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,7.9250,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [81]:
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
count,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,...,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000
mean,0.266752,2.554563,0.253989,29.551434,0.414805,0.246331,22.806457,0.001914,0.001914,0.001276,...,0.001914,0.001914,0.005743,0.001276,0.002553,0.000638,0.001276,0.156350,0.131461,0.710913
std,0.442403,0.730246,0.435430,10.087300,1.208837,0.666147,39.713131,0.043727,0.043727,0.035714,...,0.043727,0.043727,0.075592,0.035714,0.050475,0.025262,0.035714,0.363303,0.338012,0.453483
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,28.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,8.050000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,1.000000,29.699118,0.000000,0.000000,21.852100,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [82]:
y = data['Survived'].as_matrix()

In [83]:
X =data.iloc[:,1:].values

In [84]:
print ("X shape is: ", X.shape, "Y shape is" , y.shape, )

X shape is:  (1567, 839) Y shape is (1567,)


In [85]:
##make sure there are no nan values
np.any(np.isnan(X)), np.any(np.isnan(y))

(False, False)

In [86]:
## from Mike's code along http://nbviewer.jupyter.org/gist/gcr/48a6593835044e92c780fc1922df0492
pre = preprocessing.StandardScaler()
pre.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [87]:
pre.transform(X)

array([[ 0.61017669, -0.58349116, -0.748847  , ..., -0.43049421,
        -0.38904922,  0.63768545],
       [-2.12949918,  1.71382202,  0.83781221, ...,  2.32291161,
        -0.38904922, -1.56817128],
       [ 0.61017669,  1.71382202, -0.3521822 , ..., -0.43049421,
        -0.38904922,  0.63768545],
       ..., 
       [ 0.61017669,  1.71382202,  0.01464525, ..., -0.43049421,
        -0.38904922,  0.63768545],
       [-2.12949918, -0.58349116, -0.3521822 , ...,  2.32291161,
        -0.38904922, -1.56817128],
       [ 0.61017669, -0.58349116,  0.24281501, ..., -0.43049421,
         2.57036887, -1.56817128]])

In [88]:
## Divide data into train and test
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.25, random_state=42)

In [89]:
clf = LogisticRegression(C=100)
clf.fit((X_train),y_train)
clf.score((X_test),y_test)

0.89540816326530615

In [90]:
## from Mike's code along http://nbviewer.jupyter.org/gist/gcr/48a6593835044e92c780fc1922df0492
## find best C
def evaluate(C):
    return LogisticRegression(C=C) \
        .fit(pre.fit_transform(X_train), y_train) \
        .score(pre.transform(X_test), y_test)

In [91]:
def bestModel(allCs):
    """ Iterates through all of the C values we pass, and returns a tuple of the score of the best C,
    and the best C"""
    score = 0
    c_val = .001
    for c in allCs:
        currScore = evaluate(c)
        if currScore > score:
            score = currScore
            c_val = c
    return score, c_val

In [92]:
## We see that the best performing C is 100, which I have used above
possible_Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
bestC = bestModel(possible_Cs)
print(bestC)

(0.90561224489795922, 100)


In [93]:
##gives a realistic depiction of how the model will perform using cross validation
scores = cross_validation.cross_val_score(clf, X, y, cv=5)
scores.mean()

0.8984263753855336

In [94]:
##uses K fold to gather scores for each fold
def evaluate(fold):
    # Input: A tuple of (train, test) indices
    train, test = fold
    return LogisticRegression() \
        .fit(X[train], y[train]) \
        .score(X[test], y[test])

all_folds = cross_validation.KFold(len(X), n_folds=10)

In [95]:
## mean scores are 
scores = []
for fold in all_folds:
    scores.append(evaluate(fold))
mean(scores)

0.89009064184223408

In [99]:
clf_full = LogisticRegression(C=100)
clf_full.fit(X , y)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [101]:
## save model to load later
from sklearn.externals import joblib
joblib.dump(clf_full, 'log_reg_full.pkl') 

['log_reg_full.pkl',
 'log_reg_full.pkl_01.npy',
 'log_reg_full.pkl_02.npy',
 'log_reg_full.pkl_03.npy']

In [105]:
## Load model
clf2_full = joblib.load('log_reg_full.pkl') 

In [109]:
clf2_full.predict(X[0:1])

array([0])